In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import nltk
import string
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from tqdm import tqdm

In [2]:
finaldf=pd.read_csv('./final_december_df.csv')
listings=pd.read_csv('./dec_listings.csv.gz')

/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (61,62,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(listings.shape)
print(finaldf.shape)

(50599, 106)
(1254646, 10)


In [4]:
finaldf.Dominant_Topic.unique()

array([2., 3., 0., 1., 4.])

### TOPICS ARE
    0:Aparment Amenities
    1:Host Experience
    2:Location
    3:Host Communication
    4:Overall Experience

In [5]:
#we only want to segregate topics 1,2,5
topics = [0,3,4]
filtered=finaldf[finaldf.Dominant_Topic.isin(topics)]
filtered.head()

,listing_id,id,date,reviewer_id,reviewer_name,translated,Dominant_Topic,Topic_Perc_Contrib,Keywords,sentiment_score
1,2595,19176,2009-12-05,53267,Cate,Great experience.,3.0,0.2857,"place, great, stay, location, nice, good, host...",0.6249
2,2595,19760,2009-12-10,38960,Anita,I've stayed with my friend at the Midtown Cast...,3.0,0.3274,"place, great, stay, location, nice, good, host...",0.9248
3,2595,34320,2010-04-09,71130,Kai-Uwe,"We've been staying here for about 9 nights, en...",3.0,0.3450,"place, great, stay, location, nice, good, host...",0.9229
4,2595,46312,2010-05-25,117113,Alicia,We had a wonderful stay at Jennifer's charming...,0.0,0.4837,"apartment, clean, recommend, comfortable, well...",0.9464
5,2595,1238204,2012-05-07,1783688,Sergey,Hi to everyone!\nWould say our greatest compli...,0.0,0.2470,"apartment, clean, recommend, comfortable, well...",0.9869


In [6]:
#sort out the negative sentiments this 
finalfiltered=filtered[filtered['sentiment_score']>=0]

In [7]:
finalfiltered.listing_id.nunique()  #we have 36181  positive unique listings 

36181

In [8]:
finalfiltered.to_csv('./finalfitered.csv',index=False)

In [15]:
indexes=[]
for i in finalfiltered.listing_id:
    if i not in indexes:
        indexes.append(i)
print(len(indexes))

36181


In [16]:
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,20191204162729,2019-12-07,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...","- Spacious (500+ft²), immaculate and nicely fu...","Beautiful, spacious skylit studio in the heart...",none,Centrally located in the heart of Manhattan ju...,...,f,f,strict_14_with_grace_period,t,t,1,1,0,0,0.39
1,3831,https://www.airbnb.com/rooms/3831,20191204162729,2019-12-07,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,none,Just the right mix of urban center and local n...,...,f,f,moderate,f,f,1,1,0,0,4.67
2,5099,https://www.airbnb.com/rooms/5099,20191204162729,2019-12-06,Large Cozy 1 BR Apartment In Midtown East,My large 1 bedroom apartment has a true New Yo...,I have a large 1 bedroom apartment centrally l...,My large 1 bedroom apartment has a true New Yo...,none,My neighborhood in Midtown East is called Murr...,...,f,f,moderate,t,t,1,1,0,0,0.60
3,5121,https://www.airbnb.com/rooms/5121,20191204162729,2019-12-06,BlissArtsSpace!,NaN,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...,none,NaN,...,f,f,strict_14_with_grace_period,f,f,1,0,1,0,0.38
4,5178,https://www.airbnb.com/rooms/5178,20191204162729,2019-12-05,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,"You will use one large, furnished, private roo...",Please don’t expect the luxury here just a bas...,none,"Theater district, many restaurants around here.",...,f,f,strict_14_with_grace_period,f,f,1,0,1,0,3.52


In [17]:
listings.shape

(50599, 106)

In [18]:
#finally use this to filter out our listings dataset
listings_filtered=listings[listings.id.isin(indexes)]
print(listings_filtered.shape)

(36181, 106)


In [19]:
listings_filtered.to_csv('listings_filtered.csv')

In [ ]:
listings_sentiments=listings_filtered.groupby(['id','Dominant_Topic']).agg({'id':'count', 'sentiment_score':'mean'})